In [1]:
!pip install beautifulsoup4 requests openai


In [6]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

def get_chart_url(date=None):
    """
    Returns the Billboard Hot 100 URL.
    If date is None, returns current chart URL.
    If date is a datetime object, formats it into the URL.
    """
    if date is None:
        return 'https://www.billboard.com/charts/hot-100/'
    return f"https://www.billboard.com/charts/hot-100/{date.strftime('%Y-%m-%d')}/"

def scrape_billboard_chart(url):
    """
    Scrapes Billboard Hot 100 chart from the given URL.
    Returns a list of dicts with 'rank', 'title', and 'artist'.
    """
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    titles = [tag.get_text(strip=True) for tag in soup.select("li ul li h3#title-of-a-story")]
    artists = [tag.get_text(strip=True) for tag in soup.select("li ul li span.c-label.a-no-trucate")]

    songs = [{"rank": i+1, "title": titles[i], "artist": artists[i]}
             for i in range(min(len(titles), len(artists)))]
    return songs

# Automated date generation
today = datetime.today()
last_week = today - timedelta(days=7)

# Get URLs
current_url = get_chart_url()
last_week_url = get_chart_url(last_week)

# Scrape charts
print("📅 Scraping Billboard Hot 100...")
current_chart = scrape_billboard_chart(current_url)
last_week_chart = scrape_billboard_chart(last_week_url)

# Print top 5 as preview
print("\n🎶 Current Week - Top 5:")
for song in current_chart[:5]:
    print(f"{song['rank']}. {song['title']} by {song['artist']}")

print("\n🎶 Last Week - Top 5:")
for song in last_week_chart[:5]:
    print(f"{song['rank']}. {song['title']} by {song['artist']}")



📅 Scraping Billboard Hot 100...

🎶 Current Week - Top 5:
1. Luther by Kendrick Lamar & SZA
2. Die With A Smile by Lady Gaga & Bruno Mars
3. A Bar Song (Tipsy) by Shaboozey
4. Just In Case by Morgan Wallen
5. Pink Pony Club by Chappell Roan

🎶 Last Week - Top 5:
1. Luther by Kendrick Lamar & SZA
2. Evil J0rdan by Playboi Carti
3. Die With A Smile by Lady Gaga & Bruno Mars
4. Rather Lie by Playboi Carti & The Weeknd
5. The Giver by Chappell Roan


In [10]:
import openai
import json

# ⛔ Replace with your OpenAI API key
openai.api_key = ""

def gpt_chart_summary(current_chart, last_chart):
    prompt = f"""
You are a music analyst helping summarize Billboard chart trends.

Here is the current week's Billboard Hot 100 chart (top 100 songs):
{json.dumps(current_chart[:100], indent=2)}

Here is last week's chart:
{json.dumps(last_chart[:100], indent=2)}

Compare the two and tell me:
1. What are the biggest changes in ranks?
2. Any songs that entered the chart this week?
3. Which artists have multiple songs?
4. Any trends in genres or popularity?

Return the answer as a summary with bullet points.
"""

    response = openai.chat.completions.create(
        model="gpt-4",  # You can use "gpt-3.5-turbo" too
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7
    )
    return response.choices[0].message.content

# 🧠 Run GPT analysis
summary = gpt_chart_summary(current_chart, last_week_chart)
print("🎧 GPT Analysis Summary:\n")
print(summary)



🎧 GPT Analysis Summary:

- Biggest Changes in Ranks: "Evil J0rdan" by Playboi Carti dropped from rank 2 to 27, "Rather Lie" by Playboi Carti & The Weeknd went from rank 4 to 25, "The Giver" by Chappell Roan dropped from rank 5 to 29, "Beautiful Things" by Benson Boone jumped from rank 15 to 8, and "Good Credit" by Playboi Carti & Kendrick Lamar fell from rank 17 to 60.

- New Entries: "Just In Case" by Morgan Wallen, "I'm A Little Crazy" by Morgan Wallen, "Call Me When You Break Up" by Selena Gomez, benny blanco & Gracie Abrams, "Mona Lisa" by j-hope, "EOO" by Bad Bunny, "Te Queria Ver" by Aleman X Neton Vega, "This Town's Been Too Good To Us" by Dylan Scott, "How Does It Feel To Be Forgotten" by Selena Gomez & benny blanco, and "Voy A Llevarte Pa PR" by Bad Bunny entered the chart this week.

- Multiple Songs Artists: Kendrick Lamar, Playboi Carti, Morgan Wallen, Lady Gaga, Bruno Mars, Chappell Roan, Sabrina Carpenter, Teddy Swims, Drake, The Weeknd, Doechii, Billie Eilish, Benson Boo

In [15]:
top_10_songs = current_chart[:10]  # Just the top 10


In [16]:


def classify_top_10_genres(songs):
    prompt = f"""
You're a music expert. Classify the following Billboard Top 10 songs into genres like Pop, Rap, Hip-Hop, R&B, Rock, Country, EDM, Latin, etc.

Return the result as a JSON list with each item having: title, artist, genre.

Songs:
{json.dumps(songs, indent=2)}
"""

    response = openai.chat.completions.create(
        model="gpt-4",  # Or "gpt-3.5-turbo"
        messages=[{"role": "user", "content": prompt}],
        temperature=0.4
    )

    return json.loads(response.choices[0].message.content)


In [17]:
genre_predictions = classify_top_10_genres(top_10_songs)


In [18]:
for song in genre_predictions:
    print(f"{song['title']} by {song['artist']} → Genre: {song['genre']}")


Luther by Kendrick Lamar & SZA → Genre: Hip-Hop/R&B
Die With A Smile by Lady Gaga & Bruno Mars → Genre: Pop
A Bar Song (Tipsy) by Shaboozey → Genre: Rap
Just In Case by Morgan Wallen → Genre: Country
Pink Pony Club by Chappell Roan → Genre: Pop
Lose Control by Teddy Swims → Genre: R&B
Nokia by Drake → Genre: Hip-Hop/Rap
Beautiful Things by Benson Boone → Genre: Pop
Not Like Us by Kendrick Lamar → Genre: Hip-Hop/Rap
APT. by ROSE & Bruno Mars → Genre: Pop
